In [ ]:
!pip install transformers
!pip install sentence-transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import csv
from scipy import stats
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

In [ ]:
device = 'cuda'

In [ ]:
SINGLE_TRAIN_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv"
SINGLE_TEST_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv"
MULTI_TRAIN_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_multi_train.tsv"
MULTI_TEST_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_multi_test.tsv"

In [ ]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
def prepare_dataset(TRAIN_DATAPATH, TEST_DATAPATH):
    df_train = pd.read_csv(TRAIN_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
    df_test = pd.read_csv(TEST_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
    df_train['complexity'] = df_train['complexity'].astype(float)
    df_test['complexity'] = df_test['complexity'].astype(float)
    train_input = [i for i in df_train['sentence']]
    test_input = [i for i in df_test['sentence']]
    labels = [i for i in df_train['complexity']]
    test_labels = [i for i in df_test['complexity']]
    train_emb = model.encode(train_input)
    test_emb = model.encode(test_input)
    return train_emb, test_emb, labels, test_labels

In [ ]:
class NN(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.linear1 = nn.Linear(input_dim, 1536)
    self.linear2 = nn.Linear(1536, 3072)
    self.linear3 = nn.Linear(3072, 3072)
    self.linear4 = nn.Linear(3072, 1536)
    self.linear5 = nn.Linear(1536, 768)
    self.linear6 = nn.Linear(768, 768)
    self.linear7 = nn.Linear(768, 768)
    self.linear8 = nn.Linear(768, 256)
    self.linear9 = nn.Linear(256, 128)
    self.linear10 = nn.Linear(128, 64)
    self.linear11 = nn.Linear(64, 1)

  def forward(self, input):                                      # gelu or elu in initial layers is quite good, gives 0.5 r
    out = F.gelu(self.linear1(input))
    out = F.gelu(self.linear2(out))
    out = F.gelu(self.linear3(out))
    out = F.gelu(self.linear4(out))
    out = F.gelu(self.linear5(out))
    out = F.gelu(self.linear6(out))
    out = F.gelu(self.linear7(out))
    out = F.gelu(self.linear8(out))
    out = F.gelu(self.linear9(out))
    out = F.gelu(self.linear10(out))
    out = F.sigmoid(self.linear11(out))
    out = torch.squeeze(out)
    return out

In [ ]:
def pearson_loss(target, output):
    eps = 0.000001
    output_mean = torch.mean(output)
    target_mean = torch.mean(target)
    x = output - output_mean.expand_as(output)
    y = target - target_mean.expand_as(target)
    pearson = torch.dot(x, y)/ ((torch.std(output) + eps) * (torch.std(target) + eps))
    loss = (-1.0 * pearson / len(target))
    return loss

In [ ]:
input_dim = 768
print("++++Input Dimension of NN: " + str(input_dim))
nn_num_epochs = 1500
nn_model = NN(input_dim)
nn_model.to(device)
nn_optimizer = optim.Adam(nn_model.parameters(), lr = 0.00001)

In [ ]:
def train_nn(nn_model, input):
    nn_model.train()
    nn_optimizer.zero_grad()
    output = nn_model(input)                       
    loss = pearson_loss(labels, output)
    loss.backward()
    nn_optimizer.step()
    return loss.item()

In [ ]:
def test_nn(nn_model, input):
    nn_model.eval()
    with torch.no_grad():
        output = nn_model(input)
        return output

In [ ]:
def calculate_metrics(y, y_hat):
    vx = y.astype(float)
    vy = y_hat.astype(float)
    pearsonR = np.corrcoef(vx, vy)[0, 1]
    spearmanRho = stats.spearmanr(vx, vy)
    MSE = np.mean((vx - vy) ** 2)
    MAE = np.mean(np.absolute(vx - vy))
    RSquared = (pearsonR ** 2)

    print("Pearson's R: {}".format(pearsonR))
    print("Spearman's rho: {}".format(spearmanRho))
    print("R Squared: {}".format(RSquared))
    print("MSE: {}".format(MSE))
    print("MAE: {}".format(MAE))

In [ ]:
train_emb, test_emb, labels, test_labels = prepare_dataset(SINGLE_TRAIN_DATAPATH, SINGLE_TEST_DATAPATH)
nn_input = torch.tensor(train_emb, device = device, requires_grad = True)
labels = torch.tensor(labels, dtype = torch.float32, device = device, requires_grad = True)
nn_input_test = torch.tensor(test_emb, device = device)

In [ ]:
print("++++++Running for single++")
for epoch in range(nn_num_epochs):
    nn_train_loss = train_nn(nn_model, nn_input)
    print("Epoch {} : {}".format(epoch + 1, nn_train_loss))
    output = test_nn(nn_model, nn_input_test)
    print("------Metrics for test-----")
    calculate_metrics(np.array(test_labels), np.array(output.tolist()))

In [ ]:
train_emb, test_emb, labels, test_labels = prepare_dataset(MULTI_TRAIN_DATAPATH, MULTI_TEST_DATAPATH)
nn_input = torch.tensor(train_emb, device = device, requires_grad = True)
labels = torch.tensor(labels, dtype = torch.float32, device = device, requires_grad = True)
nn_input_test = torch.tensor(test_emb, device = device)

In [ ]:
print("++++++Running for multi+++")
for epoch in range(nn_num_epochs):
    nn_train_loss = train_nn(nn_model, nn_input)
    print("Epoch {} : {}".format(epoch + 1, nn_train_loss))
    output = test_nn(nn_model, nn_input_test)
    print("------Metrics for test-----")
    calculate_metrics(np.array(test_labels), np.array(output.tolist()))